In [339]:
using LinearAlgebra, SparseArrays

In [340]:
using LowLevelFEM
gmsh.initialize()

gmsh.open("cube1.geo")

Info    : Reading 'cube1.geo'...
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 10%] Meshing curve 2 (Line)
Info    : [ 20%] Meshing curve 3 (Line)
Info    : [ 30%] Meshing curve 4 (Line)
Info    : [ 40%] Meshing curve 5 (Line)
Info    : [ 50%] Meshing curve 6 (Line)
Info    : [ 60%] Meshing curve 7 (Line)
Info    : [ 60%] Meshing curve 8 (Line)
Info    : [ 70%] Meshing curve 9 (Line)
Info    : [ 80%] Meshing curve 10 (Line)
Info    : [ 90%] Meshing curve 11 (Line)
Info    : [100%] Meshing curve 12 (Line)
Info    : Done meshing 1D (Wall 0.000993763s, CPU 0.000994s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 1 (Transfinite)
Info    : [ 20%] Meshing surface 2 (Transfinite)
Info    : [ 40%] Meshing surface 3 (Transfinite)
Info    : [ 60%] Meshing surface 4 (Transfinite)
Info    : [ 70%] Meshing surface 5 (Transfinite)
Info    : [ 90%] Meshing surface 6 (Transfinite)
Info    : Done meshing 2D (Wall 0.000147724s, CPU 0.000141s)
Info    : Meshing 3D

In [341]:
mat = material("cube")
prob = Problem([mat])

Problem("cube1", :Solid, 3, 3, LowLevelFEM.Material[LowLevelFEM.Material("cube", :Hooke, 200000.0, 0.3, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 115384.61538461536, 76923.07692307692, 166666.66666666666, 1.0e-7, 0.1, 1.0)], 1.0, 8, LowLevelFEM.Geometry("", "", 0, 0, nothing, nothing, nothing, nothing))

In [342]:
supp1 = displacementConstraint("left", ux=0)
supp2 = displacementConstraint("bottom", uy=0)
supp3 = displacementConstraint("rear", uz=0)

load1 = load("right", fx=1);

In [343]:
F = loadVector(prob, [load1])
showDoFResults(F, name="load");

In [344]:
resultant(F, "right")

3-element Vector{Float64}:
 1.0
 0.0
 0.0

In [345]:
λ = mat.λ
μ = mat.μ;

In [366]:
K1 = stiffnessMatrixPoisson(prob, coefficient=3μ) + gradDivMatrix(prob, coefficient=λ - μ);
K1[1:6, 1:6]

6×6 SparseMatrixCSC{Float64, Int64} with 36 stored entries:
 81196.6    3205.13  -3205.13   2136.75   1602.56   3205.13
  3205.13  81196.6   -3205.13   1602.56   2136.75   3205.13
 -3205.13  -3205.13  81196.6   -3205.13  -3205.13  -4273.5
  2136.75   1602.56  -3205.13  81196.6    3205.13   3205.13
  1602.56   2136.75  -3205.13   3205.13  81196.6    3205.13
  3205.13   3205.13  -4273.5    3205.13   3205.13  81196.6

In [367]:
K2 = stiffnessMatrixPoisson(prob, coefficient=2μ) + gradDivMatrix(prob, coefficient=λ) - curlCurlMatrix(prob, coefficient=μ)
K2[1:6, 1:6]

6×6 SparseMatrixCSC{Float64, Int64} with 36 stored entries:
  47008.5    16025.6   -16025.6   10683.8    8012.82    3205.13
  16025.6    47008.5   -16025.6    8012.82  10683.8     3205.13
 -16025.6   -16025.6    47008.5   -3205.13  -3205.13  -21367.5
  10683.8     8012.82   -3205.13  47008.5   16025.6    16025.6
   8012.82   10683.8    -3205.13  16025.6   47008.5    16025.6
   3205.13    3205.13  -21367.5   16025.6   16025.6    47008.5

In [368]:
K3 = stiffnessMatrix(prob);
K3[1:6, 1:6]

6×6 SparseMatrixCSC{Float64, Int64} with 36 stored entries:
  47008.5    16025.6   -16025.6   10683.8    8012.82    3205.13
  16025.6    47008.5   -16025.6    8012.82  10683.8     3205.13
 -16025.6   -16025.6    47008.5   -3205.13  -3205.13  -21367.5
  10683.8     8012.82   -3205.13  47008.5   16025.6    16025.6
   8012.82   10683.8    -3205.13  16025.6   47008.5    16025.6
   3205.13    3205.13  -21367.5   16025.6   16025.6    47008.5

In [326]:
(K2.A-K3.A)[1:6, 1:6]

6×6 SparseMatrixCSC{Float64, Int64} with 36 stored entries:
 -17094.0     6410.26   -6410.26    4273.5     3205.13    6410.26
   6410.26  -17094.0    -6410.26    3205.13    4273.5     6410.26
  -6410.26   -6410.26  -17094.0    -6410.26   -6410.26   -8547.01
   4273.5     3205.13   -6410.26  -17094.0     6410.26    6410.26
   3205.13    4273.5    -6410.26    6410.26  -17094.0     6410.26
   6410.26    6410.26   -8547.01    6410.26    6410.26  -17094.0

In [327]:
f = loadVector(prob, [load1]);

In [328]:
all = constrainedDoFs(prob, [displacementConstraint("cube", ux=0, uy=0, uz=0)])
bc = constrainedDoFs(prob, [supp1, supp2, supp3])
free = setdiff(all, bc);

In [329]:
u = applyBoundaryConditions(prob, [supp1, supp2, supp3]);

In [330]:
DoFs(u)[free] = K3[free, free] \ DoFs(f)[free];

In [331]:
showDoFResults(u, name="displacement")

1

In [332]:
A = (u ∘ ∇ + ∇ ∘ u) / 2

TensorField
[[5.0e-6; 1.0587911840678754e-22; … ; 0.0; -1.5000000000000013e-6;;]]

In [333]:
showElementResults(A, name="strain")

2

In [334]:
I = TensorField(prob, "cube", [1 0 0; 0 1 0; 0 0 1])
S = 2μ * A + λ * trace(A) * I;

In [335]:
showDoFResults(S, name="stress")

3

In [336]:
openPostProcessor()

-------------------------------------------------------
Version       : 4.15.0-git
License       : GNU General Public License
Build OS      : Linux64-sdk
Build date    : 19700101
Build host    : amdci7.julia.csail.mit.edu
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blossom Cairo DIntegration Dlopen DomHex Eigen[contrib] Fltk GMP Gmm[contrib] Hxt Jpeg Kbipack LinuxJoystick MathEx[contrib] Mesh Metis[contrib] Mmg Mpeg Netgen Nii2mesh ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR TinyXML2[contrib] Untangle Voro++[contrib] WinslowUntangler Zlib tinyobjloader
FLTK version  : 1.3.8
OCC version   : 7.9.2
Packaged by   : root
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------


In [337]:
gmsh.finalize()